# Data augmentation

## Ideas
- Change background color
- Image normalization
- zoom range (1.0, 1.2)
- width shift range (10%), height shift range (10%)
- rotation range (±10°)
- brightness range (0.4 1.2)
- blurring images using averaging, median, and gaussian for the dynamic sign.
- morphological operations erosion and dilation, adding noise salt and paper, and sharpening the dataset's images.

## Imports

In [1]:
import cv2
import os
import cvzone
from cvzone.SelfiSegmentationModule import SelfiSegmentation

In [2]:
import numpy as np

In [3]:
from pathlib import Path

## Change background color

In [4]:
# BGR background colors

# usual background colors
light_beige = (201,227,234)
beige = (150,189,210)
dark_beige = (126,174,200)
white_ish = (230,231,233)
white_to_gray = (194,195,197)
blue_gray = (193,189,178)
naturalwhite = (238,240,240)

# unsual
white = (255,255,255)
orange = (33,102,243)
blue = (188,87,35)
yellow = (35,237,252)
red = (39,33,213)

In [5]:
# train_or_test = 'train'
train_or_test = 'videos'
color = 'usual_colors'

video_source_path = Path(f"D:/Documentos/Polito/Thesis/Datasets/A3LIS-147_italian/trimmed-life/{train_or_test}/")
video_destination_path = Path(f"D:/Documentos/Polito/Thesis/Datasets/A3LIS-147_italian/trimmed-life/augmented/{color}/{train_or_test}/")

video_destination_path.mkdir(parents=True, exist_ok=True)

In [6]:
usual_colors = {'beige':beige, 'lightbeige':light_beige, 'darkbeige':dark_beige, 'naturalwhite':naturalwhite, 'whiteish':white_ish, 'whitetogray':white_to_gray, 'bluegray': blue_gray}
unusual_colors = {'white':white, 'orange':orange, 'blue':blue, 'yellow':yellow, 'red':red}

### Test

#### cvzone

In [ ]:
video_folder_path = "D:/Documentos/Polito/Thesis/Datasets/A3LIS-147_italian/trimmed-10/train/"
# video_file_name = "fal_acqua (online-video-cutter.com).mp4"
# video_file_name = "mmr_allegare (online-video-cutter.com).mp4"
video_file_name = "mdp_acqua (online-video-cutter.com).mp4"

width = 720
height = 576
size = (width, height)
fps = 25
color_key = 'beige'

cap = cv2.VideoCapture(os.path.join(video_folder_path, video_file_name))
segmentor = SelfiSegmentation()
img_array = list()
success = True
while success == True:
    success, image = cap.read()
    if success:
        img_out = segmentor.removeBG(image, usual_colors[color_key], cutThreshold=0.95)
        cv2.imshow("image out", img_out)
        cv2.waitKey(200)
        img_array.append(img_out)

out = cv2.VideoWriter('test_cut_095.mp4',cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()
cv2.destroyAllWindows()
cap.release()

#### cv2

In [12]:
video_folder_path = "D:/Documentos/Polito/Thesis/Datasets/A3LIS-147_italian/trimmed-10/train/"
# video_file_name = "fal_acqua (online-video-cutter.com).mp4"
# video_file_name = "mmr_allegare (online-video-cutter.com).mp4"
video_file_name = "mdp_acqua (online-video-cutter.com).mp4"

width = 720
height = 576
size = (width, height)
fps = 25
color_key = 'beige'

cap = cv2.VideoCapture(os.path.join(video_folder_path, video_file_name))
segmentor = SelfiSegmentation()
img_array = list()
success = True
# while success == True:
success, image = cap.read()
if success:

    # u_green = np.array([104, 153, 70])
    # l_green = np.array([30, 30, 0])
    # rgba(75,228,109,255)
    # rgba(31,123,55,255)

    u_green = np.array([124, 246, 80])
    l_green = np.array([50, 110, 0])
    mask = cv2.inRange(image, l_green, u_green)
    cv2.imshow("mask", mask)
    cv2.waitKey()
    # for i in range(len(mask)):
    #     print(mask[i])
    res = cv2.bitwise_and(image, image, mask = mask)
    cv2.imshow("res", res)
    cv2.waitKey()

    background = np.zeros((height, width, 3), dtype=np.uint8)
    background[:, :, 0] = usual_colors[color_key][0]
    background[:, :, 1] = usual_colors[color_key][1]
    background[:, :, 2] = usual_colors[color_key][2]
    img_out = image - res
    img_out = np.where(img_out == 0, background, img_out)      

    cv2.imshow("image out", img_out)
    cv2.waitKey(200)
    img_array.append(img_out)

out = cv2.VideoWriter('test_cv2.mp4',cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()
cv2.destroyAllWindows()
cap.release()

### Dataset augmentation

In [9]:
size = (720, 576) # (width, height)
fps = 25

colors = ['beige', 'lightbeige', 'naturalwhite','whitetogray']
# color = usual_colors[color_key]

for color_key in colors:
    color = usual_colors[color_key]
    for video_file_name in video_source_path.glob('*.mp4'):
        label = str(video_file_name.name).split(' ')[0]
        new_video_file_name = f"{label}_{color_key}.mp4"
        cap = cv2.VideoCapture(str(video_file_name))
        segmentor = SelfiSegmentation()
        img_array = list()
        success = True
        while success == True:
            success, image = cap.read()
            if success:
                img_out = segmentor.removeBG(image, color, cutThreshold=0.8)
                img_array.append(img_out)

        out = cv2.VideoWriter(str(video_destination_path/new_video_file_name),cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

        for i in range(len(img_array)):
            out.write(img_array[i])
        out.release()
        cv2.destroyAllWindows()
        cap.release()

        print(f"Successfully created {new_video_file_name} \n")

Successfully created fal_abitare_beige.mp4 

Successfully created fal_acqua_beige.mp4 

Successfully created fal_affitto_beige.mp4 

Successfully created fal_banca_beige.mp4 

Successfully created fal_caldo_beige.mp4 

Successfully created fal_casa_beige.mp4 

Successfully created fal_cibo_beige.mp4 

Successfully created fal_data_beige.mp4 

Successfully created fal_freddo_beige.mp4 

Successfully created fal_interprete_beige.mp4 

Successfully created fal_inviare_sms_beige.mp4 

Successfully created fal_lingua_dei_segni_beige.mp4 

Successfully created fal_litro_beige.mp4 

Successfully created fal_mangiare_beige.mp4 

Successfully created fal_posta_beige.mp4 

Successfully created fal_telefono_beige.mp4 

Successfully created fef2_idle_beige.mp4 

Successfully created fef_abitare_beige.mp4 

Successfully created fef_acqua_beige.mp4 

Successfully created fef_affitto_beige.mp4 

Successfully created fef_banca_beige.mp4 

Successfully created fef_caldo_beige.mp4 

Successfully created

In [ ]:
img_array = []
for filename in video_path.glob('*.jpg'):
    img = cv2.imread(str(filename))
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

print(size)

out = cv2.VideoWriter('test.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 12, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

### Combining datasets

#### Renaming

In [3]:
train_folder_path = Path('D:/Documentos/Polito/Thesis/Datasets/A3LIS-147_italian/trimmed-10/augmented/usual_colors/hei-train-v1_v2/left/')

In [33]:
import re


In [44]:
pattern = re.compile("beige_Left_v2.jpg")

In [48]:
if re.search("beige_Left", "fal_abbonamento_beige_Left.jpg"):
    print('True')

True


In [ ]:
if pattern.match("fal_abbonamento_beige_Left.jpg"):

    print('True')

In [ ]:
pattern = re.compile("FILE\d+TEST.txt")
dir = "/test/"
for filepath in os.listdir(dir):
    if pattern.match(filepath):

In [4]:
for hei_path in train_folder_path.glob("*/*.jpg"):
    name, extension = hei_path.name.split(".")
    print(f"{hei_path.name}")
    new_path = str(hei_path.parent / f"{name}_v2.{extension}")
    hei_path.rename(new_path)
    print(new_path)

1734483201520154900_fal_abbonamento_beige_Left.jpg
D:\Documentos\Polito\Thesis\Datasets\A3LIS-147_italian\trimmed-10\augmented\usual_colors\hei-train-v1_v2\left\abbonamento\1734483201520154900_fal_abbonamento_beige_Left_v2.jpg
1734483202175724400_fal_abbonamento_beige_Left.jpg
D:\Documentos\Polito\Thesis\Datasets\A3LIS-147_italian\trimmed-10\augmented\usual_colors\hei-train-v1_v2\left\abbonamento\1734483202175724400_fal_abbonamento_beige_Left_v2.jpg
1734483203651859200_fal_abbonamento_lightbeige_Left.jpg
D:\Documentos\Polito\Thesis\Datasets\A3LIS-147_italian\trimmed-10\augmented\usual_colors\hei-train-v1_v2\left\abbonamento\1734483203651859200_fal_abbonamento_lightbeige_Left_v2.jpg
1734483204248187100_fal_abbonamento_lightbeige_Left.jpg
D:\Documentos\Polito\Thesis\Datasets\A3LIS-147_italian\trimmed-10\augmented\usual_colors\hei-train-v1_v2\left\abbonamento\1734483204248187100_fal_abbonamento_lightbeige_Left_v2.jpg
1734483205089673100_fal_abbonamento_naturalwhite_Left.jpg
D:\Documentos\

In [7]:
test = Path('D:/Documentos/Polito/Thesis/Code/sign-language-recognition/preprocessing/')

In [15]:
videos = list(test.glob('*.mp4'))

In [19]:
videos[-1]

WindowsPath('D:/Documentos/Polito/Thesis/Code/sign-language-recognition/preprocessing/test_test.mp4')

In [22]:
videos[-1].name.split('.')

['test_test', 'mp4']

In [18]:
videos[-1].rename("test.mp4")

WindowsPath('test.mp4')

In [14]:
videos[0]

WindowsPath('D:/Documentos/Polito/Thesis/Code/sign-language-recognition/preprocessing/test.mp4')

In [ ]:

images[0].with_name(f"")

WindowsPath('D:/Documentos/Polito/Thesis/Datasets/A3LIS-147_italian/trimmed-10/augmented/usual_colors/hei-train-v2/left/abbonamento/fal_abbonamento_beige_Left_v2')

## Tilt (rotation)

In [13]:
def rotate_video(input_path, output_path, angle):
    # Open the input video
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 videos

    # Define output video writer
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Center of rotation (middle of the frame)
    center = (frame_width // 2, frame_height // 2)

    # Loop through frames
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Rotation matrix
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

        # Rotate the frame
        rotated_frame = cv2.warpAffine(frame, rotation_matrix, (frame_width, frame_height))

        # Write rotated frame to output video
        out.write(rotated_frame)

    # Release resources
    cap.release()
    out.release()
    print(f"Rotated video saved as {output_path}")



In [40]:
# Usage
rotate_video("test_cut_05.mp4", "test_cut_05_rotation_test.mp4", angle=-15)  # Rotate by 15 degrees

Rotated video saved as test_cut_05_rotation_test.mp4


In [15]:
video_source_folder_path = Path("D:/Documentos/Polito/Thesis/Datasets/A3LIS-147_italian/trimmed-10/augmented/usual_colors/train/")
video_destination_folder_path = Path("D:/Documentos/Polito/Thesis/Datasets/A3LIS-147_italian/trimmed-10/augmented/usual_colors/train_tilt/")

In [50]:
str(video_source_folder_path)

'D:\\Documentos\\Polito\\Thesis\\Datasets\\A3LIS-147_italian\\trimmed-10\\augmented\\usual_colors\\train'

In [51]:
for video_file_name in video_source_folder_path.iterdir():
    name, extension = video_file_name.name.split('.')
    name = name + "_tilt"

    destination_file_name =  video_destination_folder_path / ".".join([name, extension])
    angle = np.random.choice([-10,10])

    rotate_video(str(video_file_name), str(destination_file_name), angle=angle)


Rotated video saved as D:\Documentos\Polito\Thesis\Datasets\A3LIS-147_italian\trimmed-10\augmented\usual_colors\train_tilt\fal_abbonamento_beige_tilt.mp4
Rotated video saved as D:\Documentos\Polito\Thesis\Datasets\A3LIS-147_italian\trimmed-10\augmented\usual_colors\train_tilt\fal_abbonamento_lightbeige_tilt.mp4
Rotated video saved as D:\Documentos\Polito\Thesis\Datasets\A3LIS-147_italian\trimmed-10\augmented\usual_colors\train_tilt\fal_abbonamento_naturalwhite_tilt.mp4
Rotated video saved as D:\Documentos\Polito\Thesis\Datasets\A3LIS-147_italian\trimmed-10\augmented\usual_colors\train_tilt\fal_abbonamento_whitetogray_tilt.mp4
Rotated video saved as D:\Documentos\Polito\Thesis\Datasets\A3LIS-147_italian\trimmed-10\augmented\usual_colors\train_tilt\fal_abitare_beige_tilt.mp4
Rotated video saved as D:\Documentos\Polito\Thesis\Datasets\A3LIS-147_italian\trimmed-10\augmented\usual_colors\train_tilt\fal_abitare_lightbeige_tilt.mp4
Rotated video saved as D:\Documentos\Polito\Thesis\Datasets\A